In [1]:
import pandas as pd
from google.cloud import bigquery
# プロジェクトIDの指定
project_id = 'inunaki'
# authが終わっていたら普通にクライアント認証は通る
client = bigquery.Client(project=project_id)

In [2]:
df_id = pd.read_csv("data/対象ID/生活まとめ買いクーポン対象ID.csv",header=None).rename(columns={0:"customercd"})

In [3]:
# query = """
# SELECT md5cus.customercd, sales_amt, tax_amt, cnt
# FROM (  
#    SELECT  COALESCE(customercd_new, posid.customercd) AS customercd, 
#            SUM(salesamount)AS sales_amt, SUM(salescnt)AS cnt, SUM(taxamount) AS tax_amt
#    -- FROM  `inunaki.dwd_cdp_0001.dwd_cdp_idpos_detail` posid
#    FROM `inunaki.dwd_idpos_0001.dwd_idpos_detail` posid
#    LEFT JOIN `inunaki.dim_common_0001.dim_crm_customer_change_history` ii
#    ON posid.customercd = ii.customercd
#    LEFT JOIN `inunaki.dim_common_0001.dim_product` product
#    ON posid.jan = product.jan
#    WHERE salesday BETWEEN "2023-06-19" AND "2023-07-13"
#    AND posid.customercd <> "0"
#    AND organizationname = '生活'
#    GROUP BY customercd
#   ) base
#   LEFT JOIN `inunaki.dim_crm_0001.dim_crm_customer_md5` md5cus
#   ON base.customercd = md5cus.customercd_md5
# """
query = """
SELECT md5cus.customercd, organizationname, sales_amt, tax_amt, cnt
FROM (  
   SELECT  COALESCE(customercd_new, posid.customercd) AS customercd, organizationname,
           SUM(salesamount)AS sales_amt, SUM(salescnt)AS cnt, SUM(taxamount) AS tax_amt
   -- FROM  `inunaki.dwd_cdp_0001.dwd_cdp_idpos_detail` posid
   FROM `inunaki.dwd_idpos_0001.dwd_idpos_detail` posid
   LEFT JOIN `inunaki.dim_common_0001.dim_crm_customer_change_history` ii
   ON posid.customercd = ii.customercd
   LEFT JOIN `inunaki.dim_common_0001.dim_product` product
   ON posid.jan = product.jan
   WHERE salesday BETWEEN "2023-06-19" AND "2023-07-13"
   AND posid.customercd <> "0"
  --  AND organizationname = '生活'
   GROUP BY customercd, organizationname
  ) base
  LEFT JOIN `inunaki.dim_crm_0001.dim_crm_customer_md5` md5cus
  ON base.customercd = md5cus.customercd_md5
"""
#データフレームに変換
df_data = client.query(query).to_dataframe()
df_data["customercd"] = df_data["customercd"].astype(int)

In [4]:
df_data.head()

,customercd,organizationname,sales_amt,tax_amt,cnt
0,2960040505874,グロサリー,6691,6189,32
1,2960112525144,フレッシュグループ(生鮮),14883,13784,54
2,2960120911038,フレッシュグループ(生鮮),17745,16432,47
3,2970004278294,ハード,11086,10077,45
4,2960002171178,ディリーグループ,8823,8171,79


In [5]:
# 対象の顧客だけに絞る
df_taisho_data = df_data.query("organizationname=='生活'").groupby("customercd").agg({"sales_amt":"sum","tax_amt":"sum"}).merge(df_id,on="customercd",how="right").fillna(0)

In [6]:
df_taisho_data.query("tax_amt>=1000").sort_values("customercd").head(10)

,customercd,sales_amt,tax_amt
6438,2970000006365,5817.0,5332.0
5,2970000008130,1904.0,1734.0
8,2970000012069,2699.0,2454.0
4472,2970000018757,3957.0,3598.0
5944,2970000021375,2287.0,2080.0
4850,2970000026455,6673.0,6069.0
20,2970000027292,2446.0,2224.0
22,2970000032654,2297.0,2089.0
5656,2970000035846,3539.0,3222.0
24,2970000038021,2371.0,2168.0


In [7]:
query = """
WITH base AS(
  SELECT salesday, md5cus.customercd, cnt, organizationname
  FROM ( SELECT  salesday, COALESCE(customercd_new, posid.customercd) AS customercd, SUM(salescnt)AS cnt, organizationname
  FROM `inunaki.dwd_cdp_0001.dwd_cdp_idpos_detail` posid 
  LEFT JOIN `inunaki.dim_common_0001.dim_crm_customer_change_history` ii
  ON posid.customercd = ii.customercd
  LEFT JOIN `inunaki.dim_common_0001.dim_product` product
  ON posid.jan = product.jan
  WHERE salesday BETWEEN "2023-03-01" AND "2023-05-30"
  AND posid.customercd <> "0"
  AND storecd IN(8,57,67,157,158,282,472,473)
  GROUP BY salesday, customercd, organizationname
  ) base
  LEFT JOIN `inunaki.dim_crm_0001.dim_crm_customer_md5` md5cus
  ON base.customercd = md5cus.customercd_md5
 ),
-- 生活購入状況
seikatu_id AS (
  SELECT DISTINCT customercd, 1 AS seikatu_flg
  FROM base
  WHERE organizationname = '生活'
),
-- 全体の顧客
zentai_id AS (
  SELECT DISTINCT customercd
  FROM base
),
-- 生活商品を買っていない人だけに絞る
taisho_id_data AS (
  SELECT *
  FROM(
    SELECT zentai_id.customercd, COALESCE(seikatu_flg, 0) AS seikatu_flg
    FROM zentai_id
    LEFT JOIN seikatu_id
    ON zentai_id.customercd = seikatu_id.customercd
  )
  WHERE seikatu_flg = 0
),
result_data AS(
  SELECT md5cus.customercd, sales_amt, cnt, tax_amt
  FROM (  SELECT  COALESCE(customercd_new, posid.customercd) AS customercd, SUM(salesamount)AS sales_amt, SUM(salescnt)AS cnt, SUM(taxamount) AS tax_amt
          FROM `inunaki.dwd_cdp_0001.dwd_cdp_idpos_detail` posid 
          LEFT JOIN `inunaki.dim_common_0001.dim_crm_customer_change_history` ii
          ON posid.customercd = ii.customercd
          LEFT JOIN `inunaki.dim_common_0001.dim_product` product
          ON posid.jan = product.jan
          WHERE salesday BETWEEN "2023-06-19" AND "2023-07-13"
          -- AND storecd IN(8,57,67,157,158,282,472,473)
          AND posid.customercd <> "0"
          AND organizationname = '生活'
          GROUP BY customercd
  ) base2
  LEFT JOIN `inunaki.dim_crm_0001.dim_crm_customer_md5` md5cus
  ON base2.customercd = md5cus.customercd_md5
),
-- お店には来ているのか？
raiten_data AS (
    SELECT md5cus.customercd,  1 AS raiten_flg
    FROM (SELECT DISTINCT COALESCE(customercd_new, posid.customercd) AS customercd
          FROM `inunaki.dwd_cdp_0001.dwd_cdp_idpos_detail` posid 
          LEFT JOIN `inunaki.dim_common_0001.dim_crm_customer_change_history` ii
          ON posid.customercd = ii.customercd
          WHERE salesday BETWEEN "2023-06-19" AND "2023-07-13"
          AND posid.customercd <> "0"
          GROUP BY customercd 
          ) base3
  LEFT JOIN `inunaki.dim_crm_0001.dim_crm_customer_md5` md5cus
  ON base3.customercd = md5cus.customercd_md5
)
SELECT taisho_id_data.customercd, COALESCE(tax_amt,0) AS tax_amt, seikatu_flg, COALESCE(raiten_flg,0) AS raiten_flg
FROM taisho_id_data
LEFT JOIN result_data
ON result_data.customercd = taisho_id_data.customercd
LEFT JOIN raiten_data
ON raiten_data.customercd = taisho_id_data.customercd

"""
df_data_hitaisho = client.query(query).to_dataframe()

df_data_hitaisho = df_data_hitaisho.sort_values("customercd").dropna()
df_data_hitaisho["customercd"] = df_data_hitaisho["customercd"].astype(int)

In [8]:
df_data_hitaisho.head()

,customercd,tax_amt,seikatu_flg,raiten_flg
8903,2960000023738,0,0,0
17070,2960000127290,0,0,0
45578,2960000211852,2228,0,1
968,2960000214792,0,0,0
15781,2960000215454,0,0,0


In [9]:
# 対象の顧客にフラグを立てる
df_id1 = df_id.copy()
df_id1["taisho_flg"] = 1
# 対象の顧客は外す
df_hitaisho_result = df_data_hitaisho.merge(df_id1,on="customercd",how="left").fillna(0).query("taisho_flg==0")

In [10]:
df_hitaisho_result.query("tax_amt>=1000")

,customercd,tax_amt,seikatu_flg,raiten_flg,taisho_flg
2,2960000211852,2228,0,1,0.0
6,2960000215904,2633,0,1,0.0
7,2960000217786,2923,0,1,0.0
8,2960000217830,1544,0,1,0.0
9,2960000219018,1181,0,1,0.0
...,...,...,...,...,...
48200,2970008683148,8878,0,1,0.0
48201,2970008684398,6071,0,1,0.0
48202,2970008691808,3207,0,1,0.0
48208,2970008712725,9901,0,1,0.0


In [19]:
print("対象者のクリア率")
print(round(df_taisho_data.query("tax_amt>=1000")["customercd"].nunique()/df_taisho_data["customercd"].nunique(),3)*100,"%")
print("非対象者のクリア率")
print(round(df_hitaisho_result.query("tax_amt>=1000")["customercd"].nunique()/df_hitaisho_result["customercd"].nunique(),3)*100,"%")

対象者のクリア率
27.1 %
非対象者のクリア率
16.0 %


In [12]:
# 来店率について
df_data_raiten = df_data.groupby("customercd").agg({"tax_amt":"sum"}).merge(df_id,on="customercd",how="right").fillna(0)

In [28]:
## そもそもの来店率が低いかも？？
print("対象者の来店率")
print(round(df_data_raiten.query("tax_amt>=1")["customercd"].nunique()/df_data_raiten["customercd"].nunique(),4)*100,"%")
print("非対象者の来店率")
print(round(df_hitaisho_result.query("raiten_flg==1")["customercd"].nunique()/df_hitaisho_result["customercd"].nunique()*100,2),"%")

対象者の来店率
75.53 %
非対象者の来店率
61.87 %


In [14]:
df_taisho_data.query("tax_amt>=1000").sort_values("tax_amt",ascending=False).head(10)

,customercd,sales_amt,tax_amt
3279,2970005424676,53232.0,48398.0
1258,2970001497551,47582.0,43258.0
5137,2970000539115,42935.0,39052.0
3691,2970005999730,28230.0,25684.0
3941,2970006327488,27310.0,24842.0
4939,2970000689858,24232.0,22034.0
5109,2970002085535,22157.0,20265.0
5761,2970003983182,19757.0,17965.0
2001,2970002737724,19416.0,17657.0
5491,2970000491673,18725.0,17025.0


In [15]:
df_taisho_data.query("customercd==2970001196805")

,customercd,sales_amt,tax_amt
5667,2970001196805,16587.0,15080.0


In [16]:
print("対象者のクリア率")
print(df_taisho_data.query("tax_amt>=1000")["customercd"].nunique(),df_taisho_data["customercd"].nunique())
print("非対象者のクリア率")
print(df_hitaisho_result.query("tax_amt>=1000")["customercd"].nunique(), df_hitaisho_result["customercd"].nunique())

対象者のクリア率
1754 6477
非対象者のクリア率
7315 45579


In [29]:
## そもそもの来店率が低いかも？？
print("対象者の来店率")
print(df_data_raiten.query("tax_amt>=1")["customercd"].nunique(),df_data_raiten["customercd"].nunique())
print("非対象者の来店率")
print(df_hitaisho_result.query("raiten_flg==1")["customercd"].nunique(),df_hitaisho_result["customercd"].nunique())

対象者の来店率
4892 6477
非対象者の来店率
28198 45579


In [26]:
df_taisho_data.query("tax_amt>=1000")["tax_amt"].mean()

3254.19441277081

In [27]:
df_hitaisho_result.query("tax_amt>=1000")["tax_amt"].mean()

3035.7081339712918